In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import outline_site, make_symmetric_sites, combine_sites, plot_density, Site
plt.rcParams['axes.grid'] = False 

In [ ]:
# Enrichment plot colormap
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

# Identify one or more sites from enrichment plots

# Important User Settings:

In [ ]:
lipids = ["POPG", "POPE"]
leaflets = ['low', 'upp']
root = Path("/Users/ezry/Projects/ELIC_PE_PG/aggregated/")
replicas = ["rep1", "rep3", "liam"]

enrichments = None
counts = None
helices_upr, helices_lwr = get_helices(root.joinpath(replicas[2]))
for rep in replicas:
    reppath = root.joinpath(rep)
    file_list = get_file_list(reppath, lipids)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(file_list[0])
    cts, rich = read_rep(file_list, lipids, leaflets, enrich=True)
    if counts is None:
        counts = cts
        enrichments = rich
    else:
        counts = counts+cts
        enrichments = enrichments+rich
    

counts = counts/len(replicas)
enrichments = enrichments/len(replicas)
thetas = np.unique(theta.flatten())

In [ ]:
thetas = np.unique(theta.flatten())
site1def = {'theta_start':4, 'width':3, 'inner_r':24, 'outer_r':30, 'Ntheta':Ntheta, 'dr':dr, 'dth':dth, 'exrho':1, 'frames':frames, 'sitename':"Site 1 "}
site2def = {'theta_start':9, 'width':4, 'inner_r':22, 'outer_r':32, 'Ntheta':Ntheta, 'dr':dr, 'dth':dth, 'exrho':1, 'frames':frames, 'sitename':"Site 2 "}

dummy_site1 = make_symmetric_sites([], **site1def)

In [ ]:
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

In [ ]:

enrichments.at['POPE', 'low'][0:1,:]=0
fig, axes = polar_plot(enrichments, 
                       theta, 
                       radius, 
                       lipids, 
                       helices_lwr, 
                       helices_upr, 
                       colorbychain=False, 
                       vmin=0.75, 
                       vmax=1.5, 
                       vmid=1,
                       figheight=8,
                       figwidth=8,
                       cmap=my_cmap)
fig.tight_layout()
plt.savefig(root.joinpath("ELIC_enrichments.pdf"))
plt.show()

# Get Accessible Area

# Step 0: define a site

This is done with the plotting_show_binID notebook.
Some examples from ELIC:

In [ ]:
# User parameters
lipid = "DPPC"
leaf = "upp"
the_site_def = site1def
data_root = Path("/Users/ezry/Projects/ELIC_DPPC/elicPE17_DPPC/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

# Update exrho and frames based on the above since it will differ for DPPC. All other parameters are constant across systems.
the_site_def['exrho'] = exrho
the_site_def['frames'] = frames


the_sites = make_symmetric_sites(the_data, **the_site_def)
site1_symmetric = combine_sites(the_sites, exrho, f"{the_site_def['sitename'][:-1]}, Symmetric", symmetric=True)

sites = {'a_site':the_sites[0], 
         'the_site_symmetric':site1_symmetric,
         }

# Step 1 get the total area of the site in question

In [ ]:
for name, site in sites.items():
    print(f"{site.title:<20}:{np.round(site.area,1):>5}")

# Step 2 get the density 
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 125
bulk_counts_path = Path(f"/Users/ezry/Projects/Polar_Binning_DeltaG/accessible_area/counts_{area}.out")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

# Compare the bulk mode with the site mode

In [ ]:
rows = 1
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(10,5))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)

fig.tight_layout()
plt.savefig(data_root.joinpath("Raw_distributions.pdf"))

In [ ]:
print(f"{'Site Name':<20}:{'Experimental Mode':>5}")
expt_modes = {}
for key, site in sites.items():
    counts = site.counts.astype(int)
    bincounts = np.bincount(counts.flatten())[1:]
    the_mode = np.argmax(bincounts)
    expt_modes[key] = the_mode
    print(f"{key:<20}:{np.round(the_mode,1):>5}")

# The accessible area is...
ASSUMING $\left(A_\mathrm{site} \approx A_\mathrm{bulk}\right) \forall A_\mathrm{site}$

$A_\mathrm{acc} = A_\mathrm{total} \frac{M_\mathrm{site}}{M_\mathrm{bulk}}$

In [ ]:
print(f"{'Site Name':<20}:{'Accessible Area':>5}")
areas = {}
for name, site in sites.items():
    A_acc = site.area * expt_modes[name] / bulk_mode
    site.area = A_acc
    print(f"{name:<20}:{np.round(A_acc,1):>5}")

# Replot with new areas:

In [ ]:
for name, site in sites.items():
    site.Npeak = np.mean(bulk_counts)/area * site.area

In [ ]:
rows = 2
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(15,15))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)
    ax.text(.01, .99, f"Accessible area:"+str(np.round(site.area,1))+r"$\AA^2$", ha='left', va='top', transform=ax.transAxes)
    ax.vlines(np.mean(site.counts), 0, 0.3)
plt.savefig(data_root.joinpath(f"replotted_with_corrected_expectation_area{area}_{leaf}.pdf"))